In [1]:
from __future__ import absolute_import, division, print_function

import os
import re
import time
import argparse
from tqdm import tqdm, trange
import numpy as np
import pandas as pd
from scipy import interp
import matplotlib.pyplot as plt

import keras
from keras import backend as K
from keras.backend import clear_session
from keras import regularizers 
from keras.models import Sequential, Model
from keras.layers import Dense, Input, Dropout, Activation, BatchNormalization, InputLayer, Embedding, Reshape, Concatenate, Masking, TimeDistributed
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.utils.vis_utils import plot_model
from keras.utils.np_utils import to_categorical

import tensorflow as tf

import optuna
from optuna.integration import TFKerasPruningCallback
from optuna.trial import TrialState
from optuna.samplers import RandomSampler
from optuna.visualization import plot_intermediate_values
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_contour
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_slice

from sklearn.preprocessing import label_binarize
from sklearn.metrics import f1_score, confusion_matrix, roc_curve, auc, classification_report, precision_recall_curve, average_precision_score

from focal_loss import SparseCategoricalFocalLoss

from Data_helper import LoadDataImputed
from utils.layers import ExternalMasking
from utils.grud_layers import GRUD, Bidirectional_for_GRUD
from BuildModel import LoadModel
from utils.attention_function import attention_3d_block_spatial as PreAttention

Using TensorFlow backend.


In [2]:
arg_parser = argparse.ArgumentParser()
arg_parser.add_argument('--working_path', default='.')

## data
arg_parser.add_argument('--dataset_name', default='phase_viii', 
                         help='The data files should be saved in [working_path]/data/[dataset_name] directory.')
arg_parser.add_argument('--fold', type=int, default=0, 
                         help='the fold data taken to use, there are 5 folds or 10 folds')
arg_parser.add_argument('--label', default=3, type=int, choices=[-1,0,1,2,3,4],
                         help='the label type')

## model
arg_parser.add_argument('--model_type', default='LSTM', choices=['LR', 'RF', 'SVM', 'DNN', 'DNN_EE', 'GRUD', 'HMM_EF', 'HMM_LF', 'LSTM'])
arg_parser.add_argument('--max_timestep', type=int, default=200, 
                        help='Time series of at most # time steps are used. Default: 200.')
arg_parser.add_argument('--max_timestamp', type=int, default=48*60*60,choices=[120*60*60, 72*60*60, 96*60*60, 120*60*60],
                        help='Time series of at most # seconds are used. Default: 48 (hours).')
arg_parser.add_argument('--use_bidirectional_rnn', default=False)
# Train
arg_parser.add_argument('--trainORvalidation', default='Validation', choices=['Train', 'Validation'])
arg_parser.add_argument('--epochs', type=int, default=100)
arg_parser.add_argument('--early_stopping_patience', type=int, default=10)
arg_parser.add_argument('--batch_size', type=int, default=32)
# set the actual arguments if running in notebook
if not (__name__ == '__main__' and '__file__' in globals()):
    ARGS = arg_parser.parse_args([
        '--model_type', 'LSTM',
        '--dataset_name', 'phase_viii',
        '--fold', '0',
        '--epochs', '100',
        '--trainORvalidation', 'Validation'
    ])
else:
      ARGS = arg_parser.parse_args()

print('Arguments:', ARGS) 

Arguments: Namespace(batch_size=32, dataset_name='phase_viii', early_stopping_patience=10, epochs=100, fold=0, label=3, max_timestamp=172800, max_timestep=200, model_type='LSTM', trainORvalidation='Validation', use_bidirectional_rnn=False, working_path='.')


In [3]:
def get_cm(label_info, y_validation, y_pred, runtime):
    # get the confusion matrix
    C = confusion_matrix(y_validation, y_pred)
    if label_info == 'flatten multi-classification':
        df_cm = pd.DataFrame(C, columns=['0','1','2','3','4','5','6'])
    elif label_info == 'bacterial, viral and others':
        df_cm = pd.DataFrame(C, columns=['0','1','2'])
    else:
        df_cm = pd.DataFrame(C, columns=['0','1'])
    df_cm['bootstrap_index'] = runtime
    df_cm['label'] = label_info

    return df_cm
    
def save_report(label_info, y_validation, y_pred, runtime):
    report = classification_report(y_validation, y_pred,digits=5, output_dict = True)
    report = pd.DataFrame(report).transpose().reset_index()
    report['bootstrap_index'] = runtime
    report['label'] = label_info

    return report

def plot_roc(nclasses, y_validation, probas):
    '''
    label_info: The label information to tag the figure name
    nclasses: The dim of y, which determines the way of plot
    y_validation: The y value of validation dataset
    probas: The probability of model results
    '''
    # Compute ROC curve and area the curve
    if nclasses == 2:
        fpr, tpr, thresholds = roc_curve(y_validation, probas[:, 1])
        roc_auc = auc(fpr, tpr)

        roc_value = {'fpr': fpr, 'tpr': tpr, 'roc_auc': roc_auc}
    else:
        fpr = dict()
        tpr = dict()
        roc_auc = dict()
        for m in range(nclasses):
            fpr[m], tpr[m], _ = roc_curve(y_validation[:, m], probas[:, m])
            roc_auc[m] = auc(fpr[m], tpr[m])
        # Compute macro-average ROC curve and ROC area
        all_fpr = np.unique(np.concatenate([fpr[n] for n in range(nclasses)]))
        mean_tpr = np.zeros_like(all_fpr)
        for k in range(nclasses):
            mean_tpr += interp(all_fpr, fpr[k], tpr[k])
        # Finally average it and compute AUC
        mean_tpr /= nclasses
        fpr["macro"] = all_fpr
        tpr["macro"] = mean_tpr
        roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])
        # save the data
        roc_value = {'fpr': fpr, 'tpr': tpr, 'roc_auc': roc_auc}

    return roc_value

def plot_prc(nclasses, y_validation, y_probas, y_pred):
    '''
    label_info: The label information to tag the figure name
    ydim: The dim of y, which determines the way of plot
    y_validation: The y value of validation dataset
    probas: The probability of model results
    '''
    # Compute PRC curve and area the curve
    if nclasses == 2:
        precision, recall, _ = precision_recall_curve(y_validation, y_probas[:, 1])
        f1, auprc = f1_score(y_validation, y_pred), auc(recall, precision)
        
        # save the data
        prc_value = {'precision': precision, 'recall': recall, 'auprc': auprc}

    else:
        precision = dict()
        recall = dict()
        average_precision = dict()
        for i in range(nclasses):
            precision[i], recall[i], _ = precision_recall_curve(y_validation[:, i], y_probas[:, i])
            average_precision[i] = average_precision_score(y_validation[:, i], y_probas[:, i])
        
        precision["micro"], recall["micro"], _ = precision_recall_curve(y_validation.ravel(), y_probas.ravel())
        average_precision["micro"] = average_precision_score(y_validation, y_probas, average="micro")

        # save the data
        prc_value = {'precision': precision, 'recall': recall, 'auprc': average_precision}
    
    return prc_value

In [4]:
# define the label, -1 represent the multi-class classification
model_path = os.path.join(ARGS.working_path, 'model_tuning', 'phase_viii', '48hours', '20230610', 'LSTM_7dim_48hrs_mean2_Paper(optuna)')
if not os.path.exists(model_path):
    os.makedirs(model_path)
output_path = os.path.join(ARGS.working_path, 'test', 'phase_viii', '48hours', '20230610', 'LSTM_7dim_48hrs_mean2_Paper(optuna)')
if not os.path.exists(output_path):
    os.makedirs(output_path)

T = time.strftime("%Y%m%d%H%M%S", time.localtime())
LABEL_DICT = {'-1':'flatten multi-classification', 
              '0': 'infectious and non-infectious', 
              '1':'bacterial, viral and others', 
              '2':'NIID and tumor', 
              '3': 'AD and AID',
              '4':'HM and SM'}
label_type = [1, 0, 2, 3, 4]
all_avg = {}
for label in label_type:
    
    # Load the data
    dataset = LoadDataImputed(data_path=os.path.join('.', 'data', 'phase_viii', '48hours', 'processed', 'imputed_mean_48_2/data4hc_v20220401'), 
                        model_type=ARGS.model_type,
                        label_name=label,
                        max_timestep=ARGS.max_timestep,
                        max_timestamp=ARGS.max_timestamp)

    # Load the data
    X_train, y_train, nclasses_train, folds_train, shapes_train = dataset.training_generator(ARGS.fold)
    X_validation, y_validation, nclasses_validation,folds_validation, shapes_validation = dataset.validation_generator(ARGS.fold)
    X_test, y_test, nclasses_test, folds_test, shapes_test = dataset.test_generator(ARGS.fold)
    print(nclasses_train)
    # hyperparameter tuning
    print("[Info]: The LSTM model test of {0} is processing".format(LABEL_DICT[str(label)]))
    #take the best model
    dir_model = []
    idx_model = np.array([])
    for i in os.listdir(model_path):
        if LABEL_DICT[str(label)] in i and '.h5' in i:
            dir_model.append(i)
    if len(dir_model) == 0:
        continue
    for j in dir_model:
        seachobj = re.search(r"\d+(?=\.h5)", j)
        idx_model = np.append(idx_model, int(seachobj.group()))
    target_model = dir_model[np.argmax(idx_model)]
    # load the model
    print("[Info]: The Best model of ({0}) is {1}".format(LABEL_DICT[str(label)], target_model))
    model = LoadModel(os.path.join(model_path, target_model))

    # set bootstrap on test dataset
    if label == -1:
        df_cms = pd.DataFrame(columns=['bootstrap_index', 'label', '0','1','2','3','4','5','6'])
    elif label == 1:
        df_cms = pd.DataFrame(columns=['bootstrap_index', 'label', '0','1','2'])
    else:
        df_cms = pd.DataFrame(columns=['bootstrap_index', 'label', '0','1'])
    df_reports = pd.DataFrame(columns=['bootstrap_index', 'label', 'index', 'precision', 'recall', 'f1-score', 'support'])
    npy_roc = {}
    npy_prc = {}
    for runtime in tqdm(range(1000)):
        # print("[Info]: The index of bootstrap is {}".format(runtime))
        idx = np.random.choice(len(X_test), size=len(X_test), replace=True)
        X_test_boot = X_test[idx]
        y_test_boot = y_test[idx]

        # predict the test dataset
        y_pred_proba = model.predict(X_test_boot)
        y_pred = np.argmax(y_pred_proba, axis=1)

        # plot the confusion matrix
        df_cm = get_cm(LABEL_DICT[str(label)], y_test_boot, y_pred, runtime)
        df_cms = pd.concat([df_cms, df_cm], axis=0, ignore_index=True)

        # save the test dataset results
        df_report = save_report(LABEL_DICT[str(label)], y_test_boot, y_pred, runtime)
        df_reports = pd.concat([df_reports, df_report], axis=0, ignore_index=True)

        # # plot the ROC and PRC curve
        if label == -1:
            y_test_one_hot = label_binarize(y_test_boot, classes=[0,1,2,3,4,5,6])
            roc_dict = plot_roc(nclasses_train, y_test_one_hot, y_pred_proba)
            prc_dict = plot_prc(nclasses_train, y_test_one_hot, y_pred_proba, y_pred)
        elif label == 1:
            y_test_one_hot = label_binarize(y_test_boot, classes=[0,1,2])
            roc_dict = plot_roc(nclasses_train, y_test_one_hot, y_pred_proba)
            prc_dict = plot_prc(nclasses_train, y_test_one_hot, y_pred_proba, y_pred)
        else:
            roc_dict = plot_roc(nclasses_train, y_test_boot, y_pred_proba)
            prc_dict = plot_prc(nclasses_train, y_test_boot, y_pred_proba, y_pred)

        npy_roc[runtime] = roc_dict
        npy_prc[runtime] = prc_dict

    df_cms.to_csv(os.path.join(output_path, 'confusion_matrix(test_bootstrap)({}).csv'.format(LABEL_DICT[str(label)])))
    df_reports.to_csv(os.path.join(output_path, 'classification_reuslts(test_bootstrap)({}).csv'.format(LABEL_DICT[str(label)])))
    np.save(os.path.join(output_path, 'roc(test_bootstrap)({}).npy'.format(LABEL_DICT[str(label)])), npy_roc)
    np.save(os.path.join(output_path, 'prc(test_bootstrap)({}).npy'.format(LABEL_DICT[str(label)])), npy_prc)

    # calculate the average value
    roc_list = []
    prc_list = []
    if label in [0,2,3,4]:
        for key, value in npy_roc.items():
            roc_list.append(value['roc_auc'])

        for key, value in npy_prc.items():
            prc_list.append(value['auprc'])
    else:
        for key, value in npy_roc.items():
            roc_list.append(value['roc_auc']['macro'])

        for key, value in npy_prc.items():
            prc_list.append(value['auprc']['micro'])
    
    roc_mean = np.mean(roc_list)
    roc_std = np.std(roc_list)
    prc_mean = np.mean(prc_list)
    prc_std = np.std(prc_list)

    all_avg[LABEL_DICT[str(label)]] = {'roc_mean': roc_mean,
                                       'roc_std': roc_std,
                                       'prc_mean': prc_mean,
                                       'prc_std': prc_std}
np.save(os.path.join(output_path, 'statistics_test(LSTM).npy'), all_avg)
print("Finished!")

3
[Info]: The LSTM model test of bacterial, viral and others is processing
[Info]: The Best model of (bacterial, viral and others) is LSTM(bacterial, viral and others)_trial#32.h5


  0%|          | 0/1000 [00:00<?, ?it/s]/home/zju/anaconda3/envs/gru/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/home/zju/anaconda3/envs/gru/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/home/zju/anaconda3/envs/gru/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/home/zju/anaconda3/envs/gru/lib/python3.6/site-packages/ipykernel_launcher.py:47: DeprecationWarning:

scipy.interp is deprecated a

2
[Info]: The LSTM model test of infectious and non-infectious is processing
[Info]: The Best model of (infectious and non-infectious) is LSTM(infectious and non-infectious)_trial#88.h5


100%|██████████| 1000/1000 [34:03<00:00,  2.04s/it]


2
[Info]: The LSTM model test of NIID and tumor is processing
[Info]: The Best model of (NIID and tumor) is LSTM(NIID and tumor)_trial#68.h5


...

2
[Info]: The LSTM model test of AD and AID is processing
[Info]: The Best model of (AD and AID) is LSTM(AD and AID)_trial#74.h5


100%|██████████| 1000/1000 [04:41<00:00,  3.55it/s]


2
[Info]: The LSTM model test of HM and SM is processing
[Info]: The Best model of (HM and SM) is LSTM(HM and SM)_trial#53.h5


100%|██████████| 1000/1000 [05:27<00:00,  3.05it/s]


Finished!


In [1]:
import numpy as np
data = np.load('/mnt/data/wzx/jupyter_notebook/HC4FUO/test/phase_viii/48hours/20230610/LSTM_7dim_48hrs_mean2_Paper(optuna)/statistics_test(LSTM).npy', allow_pickle=True).tolist()
data

{'bacterial, viral and others': {'roc_mean': 0.5964261238812119,
  'roc_std': 0.013649250119008347,
  'prc_mean': 0.6996154852231523,
  'prc_std': 0.014234841658086651},
 'infectious and non-infectious': {'roc_mean': 0.6370441088621905,
  'roc_std': 0.01371636583807894,
  'prc_mean': 0.38211006619096144,
  'prc_std': 0.020157470511847946},
 'NIID and tumor': {'roc_mean': 0.48399227394619354,
  'roc_std': 0.024172105507290237,
  'prc_mean': 0.5794622551273949,
  'prc_std': 0.030471041193293382},
 'AD and AID': {'roc_mean': 0.5403530570548949,
  'roc_std': 0.042653329867437244,
  'prc_mean': 0.36658155891365096,
  'prc_std': 0.05480109958239199},
 'HM and SM': {'roc_mean': 0.7250078065012604,
  'roc_std': 0.02956308048441437,
  'prc_mean': 0.7314766040033431,
  'prc_std': 0.03868059867726256}}